<a href="https://colab.research.google.com/github/Solajungq/ta_indiv/blob/main/TA_Individual_Assignment_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment effect of artificial intelligence

# build a full RAG pipeline using Llama Index

We will install an open-source, smallish LLM Phi-3, from Microsoft, to run locally on our Colab instance to do the generation. To augment the results we will give the LLM access to our ArXiv abstracts this time stored in Chroma db as a vector store/document store. Finally, we use Ollama as an interface to interact with Phi-3 on our machine.

## Install Ollama

In [ ]:
# Install Ollama v0.1.30
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.30#' | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10091    0 10091    0     0  20112      0 --:--:-- --:--:-- --:--:-- 20141
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Next we do a bit of work to get Ollama running in the background of our Colab (Linux) instance

In [ ]:
# Setup the model as a global variable
OLLAMA_MODEL='phi:latest'

# Add the model to the environment of the operating system
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL # declare this as a global variable (our model)
!echo $OLLAMA_MODEL # print the global variable to check it saved

import subprocess
import time

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

# Use subprocess.Popen to run the command
process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

print("Process ID:", process.pid) # print the process ID
time.sleep(5)  # Makes Python wait for 5 seconds

!ollama -v # print the Ollama version number as a check

phi:latest
Process ID: 11312
ollama version is 0.1.38


## Pull the model

In [ ]:
# Query the model via the command line
# First time running it will "pull" (import) the model
!ollama run $OLLAMA_MODEL "Give me short summary of the environmental effect of artificial intelligence"

⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸  Artificial intelligence (AI) can have both positive and negative impacts on the environment. On one hand, AI has the potential to help us reduce our carbon footprint by optimizing energy usage, increasing efficiency in manufacturing processes, and reducing waste. However, the production and disposal of electronic components used in AI technology also contribute to e-waste, which can be harmful to the environment. Additionally, the energy consumption required to power AI systems can have a significant impact on greenhouse gas emissions if not properly managed. Overall, it is important to consider both the potential benefits and drawbacks of AI in order to minimize its environmental imp

## Install packages

In [ ]:
# Install prerequisites
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-chroma
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install chromadb

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Import the Ollama class
from llama_index.llms.ollama import Ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 6.9 MB/s eta 0:00:00


## Connect LLM (Ollama)

In [ ]:
# Use the global variable (OLLAMA_MODEL) as our LLM
# Set a timeout of 4 minutes
llm = Ollama(model=OLLAMA_MODEL, request_timeout=240.0)

## Prompt template

In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Always answer the question, even if the context isn't helpful."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# Augment with data

Scopus abstract search 237 results

- search keyword: effect AND of AND the AND development AND of AND artificial AND intelligence AND on AND climate AND change
- Language: limited to English
- search date: 21/05/2024
- Export date and time: 21/05/2024 14:36

https://www2.scopus.com/results/results.uri?sort=plf-f&src=s&sid=73addfdc08abf58dbc728d0749640fa3&sot=a&sdt=a&cluster=scolang%2C%22English%22%2Ct&sl=85&s=TITLE-ABS-KEY%28Effect+of+the+development+of+Artificial+intelligence+on+climate+change%29&origin=searchhistory&txGid=8a496034415cafb6c2928271e177f4bb&sessionSearchId=73addfdc08abf58dbc728d0749640fa3&limit=10

In [1]:
# import data from the repository
import pandas as pd

# Define the URL of the raw CSV file on GitHub
url = "https://raw.githubusercontent.com/Solajungq/ta_indiv/main/scopus.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(url)

# Display the first few rows of the DataFrame
df.head()

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,Page count,Cited by,DOI,Link,Abstract,Document Type,Publication Stage,Open Access,Source,EID
0,Ahmed A.; He P.; He P.; Wu Y.; He Y.; Munir S.,"Ahmed, Ayesha (57214602368); He, Pengfei (5531...",57214602368; 55316705800; 55599430200; 5574991...,Environmental effect of agriculture-related ma...,2023,Environment International,173,NaN,107819,NaN,...,NaN,9,10.1016/j.envint.2023.107819,https://www.scopus.com/inward/record.uri?eid=2...,Agriculture-related manufactured nano-objects ...,Review,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85149063433
1,Rodríguez-Alonso C.; Pena-Regueiro I.; García Ó.,"Rodríguez-Alonso, Carlos (58933063400); Pena-R...",58933063400; 58932392800; 57613001000,Digital Twin Platform for Water Treatment Plan...,2024,Sensors,24,5,1568,NaN,...,NaN,0,10.3390/s24051568,https://www.scopus.com/inward/record.uri?eid=2...,The effects of climate change and the rapid gr...,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85187444087
2,Chen B.; Madni G.R.,"Chen, Bing (58701784000); Madni, Ghulam Rasool...",58701784000; 57202096035,Achievement of sustainable environment through...,2023,PLoS ONE,18,11 November,e0292403,NaN,...,NaN,3,10.1371/journal.pone.0292403,https://www.scopus.com/inward/record.uri?eid=2...,Social media is emerged as a powerful communic...,Article,Final,All Open Access; Gold Open Access; Green Open ...,Scopus,2-s2.0-85177235356
3,Marzouk M.; Azab S.,"Marzouk, Mohamed (7005098184); Azab, Shimaa (5...",7005098184; 55960276800,Modeling climate change adaptation for sustain...,2024,Environmental Monitoring and Assessment,196,2,147,NaN,...,NaN,0,10.1007/s10661-023-12287-2,https://www.scopus.com/inward/record.uri?eid=2...,The world is currently confronting one of its ...,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-85182220000
4,Ekwueme B.N.,"Ekwueme, Benjamin Nnamdi (57221540760)",57221540760,Deep neural network modeling of river discharg...,2024,Earth Science Informatics,17,2,NaN,1161,...,16.0,0,10.1007/s12145-023-01219-w,https://www.scopus.com/inward/record.uri?eid=2...,Precise forecast of river discharge is crucial...,Article,Final,NaN,Scopus,2-s2.0-85182159382


In [ ]:
# Use Llama Index's SimpleDirectoryReader to load all the document
# Load documents
reader = SimpleDirectoryReader("/content/data_ev") # load documents from the /data folder
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Loaded 10 docs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Build Vector database

In [ ]:
# Create client ("db") and a database ("chroma_db")
db = chromadb.PersistentClient(path="./chroma_db") # chromadb: should be downloaded. Vector DB

# Create a collection/table ("demo-for-ram") in the db
chroma_collection = db.create_collection("demo-for-ram")

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# Specify Chroma as our vector db
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index
index = VectorStoreIndex.from_documents(
    docs, # the abstract documents (e.g. "Output0.txt")
    storage_context = storage_context,
    embed_model = embed_model
)

# Print the metadata
print(chroma_collection)

# Print the name of the collection (table)
print(f'Collection name is: {chroma_collection.name}')

name='demo-for-ram' id=UUID('05e12d4a-005b-4550-8ebe-66ee9348cb2a') metadata=None tenant='default_tenant' database='default_database'
Collection name is: demo-for-ram


# Query to RAG

In [ ]:
query_engine = index.as_query_engine() # use the vector db for queries

response = query_engine.query("How artificial intelligence affect the environment?") # query Phi-3 with context
response.response # print the response

ReadTimeout: timed out

In [ ]:
response.metadata # print the full output from the LLM